In [1]:
import os 
import pandas as pd
from ast import literal_eval

In [70]:
from collections import Counter

### File/dir locations

In [2]:
DATA_DIR = os.getenv("DATA_DIR")
filename = "preprocessed_taxon_pageseq_20190114_20190116.csv.gz"
# df_file = os.path.join(DATA_DIR, "processed_journey", filename)
df_rel_file = os.path.join(DATA_DIR, "processed_journey", "rel_"+filename)
df_rel_file

'/Users/felisialoukou/Documents/govuk-network-data/data/processed_journey/rel_preprocessed_taxon_pageseq_20190114_20190116.csv.gz'

### Load up unfiltered data: 14-16/01/19

In [3]:
# df = pd.read_csv(df_file, sep="\t", compression = "gzip")
# df.shape

Original dataset shape (5048130, 17)

### Prepare dataset: keep only journeys containing "relatedLinkClicked" event
Requires further filtering because eventAction need to be "Related link" to avoid the inclusion of "Explore the topic" links.

In [5]:
# df_rel = df[df.Sequence.str.contains("relatedLinkClicked")].copy(deep=True)
# df_rel.to_csv(df_rel_file, sep="\t", compression = "gzip")

### Evaluate journeys with related links

In [6]:
df_rel = pd.read_csv(df_rel_file, sep="\t", compression = "gzip")

In [10]:
df_rel.drop("Unnamed: 0",axis=1,inplace=True)

In [14]:
df_rel['Page_Seq_Occurrences'] = df_rel.groupby('PageSequence')['Occurrences'].transform('sum')

In [7]:
df_rel.shape

(592902, 18)

In [17]:
df_rel.head()

,Occurrences,DeviceCategories,PageSeq_Length,Actions_Length,Dates,Sequence,PageSequence,Page_Event_List,Page_List,Event_List,num_event_cats,Event_cats_agg,Event_cat_act_agg,Taxon_List,Taxon_Page_List,Page_List_NL,Page_Seq_NL,Page_Seq_Occurrences
0,1,"[('desktop', 1)]",19,36,"[('20190114', 1)]",/limited-company-formation<<PAGE<:<NULL<:<NULL...,/limited-company-formation>>/government/public...,"[('/limited-company-formation', 'PAGE<:<NULL<:...","['/limited-company-formation', '/government/pu...","[('PAGE_NULL', 'PAGE_NULL'), ('user_satisfacti...",7,"[('PAGE_NULL', 19), ('user_satisfaction_survey...","[(('PAGE_NULL', 'PAGE_NULL'), 19), (('user_sat...","['386180ca-4219-478e-8b80-a271dd5ce5f2,c3bc9a5...","[('/limited-company-formation', ('386180ca-421...","['/limited-company-formation', '/government/pu...",/limited-company-formation>>/government/public...,1.0
1,1,"[('desktop', 1)]",38,73,"[('20190114', 1)]",/browse/working/state-pension<<PAGE<:<NULL<:<N...,/browse/working/state-pension>>/state-pension-...,"[('/browse/working/state-pension', 'PAGE<:<NUL...","['/browse/working/state-pension', '/state-pens...","[('PAGE_NULL', 'PAGE_NULL'), ('secondLevelBrow...",12,"[('PAGE_NULL', 38), ('secondLevelBrowseLinkCli...","[(('PAGE_NULL', 'PAGE_NULL'), 38), (('secondLe...","['other', 'b90e86de-2b62-493f-b7dd-31b89ccc97f...","[('/browse/working/state-pension', ('other',))...","['/browse/working/state-pension', '/state-pens...",/browse/working/state-pension>>/state-pension-...,1.0
2,1,"[('tablet', 1)]",32,50,"[('20190114', 1)]",/browse/business/setting-up<<EVENT<:<user_sati...,/browse/business/setting-up>>/set-up-business>...,"[('/browse/business/setting-up', 'EVENT<:<user...","['/browse/business/setting-up', '/set-up-busin...","[('user_satisfaction_survey', 'banner_shown'),...",8,"[('user_satisfaction_survey', 2), ('PAGE_NULL'...","[(('user_satisfaction_survey', 'banner_shown')...","['other', '172c9d39-445f-4501-b0b2-1d527cc7cab...","[('/browse/business/setting-up', ('other',)), ...","['/browse/business/setting-up', '/set-up-busin...",/browse/business/setting-up>>/set-up-business>...,1.0
3,1,"[('tablet', 1)]",24,49,"[('20190114', 1)]",/self-assessment-tax-returns/sending-return<<P...,/self-assessment-tax-returns/sending-return>>/...,[('/self-assessment-tax-returns/sending-return...,['/self-assessment-tax-returns/sending-return'...,"[('PAGE_NULL', 'PAGE_NULL'), ('contentsClicked...",12,"[('PAGE_NULL', 24), ('contentsClicked', 3), ('...","[(('PAGE_NULL', 'PAGE_NULL'), 24), (('contents...","['104ee859-8278-406b-80cb-5727373e0198,3564cf4...",[('/self-assessment-tax-returns/sending-return...,['/self-assessment-tax-returns/sending-return'...,/self-assessment-tax-returns/sending-return>>/...,1.0
4,1,"[('desktop', 1)]",13,28,"[('20190114', 1)]",/government/organisations/hm-revenue-customs/c...,/government/organisations/hm-revenue-customs/c...,[('/government/organisations/hm-revenue-custom...,['/government/organisations/hm-revenue-customs...,"[('PAGE_NULL', 'PAGE_NULL'), ('navFinderLinkCl...",11,"[('PAGE_NULL', 13), ('navFinderLinkClicked', 1...","[(('PAGE_NULL', 'PAGE_NULL'), 13), (('navFinde...","['other', 'other', '5984199c-d85c-4ffc-b13a-c5...",[('/government/organisations/hm-revenue-custom...,['/government/organisations/hm-revenue-customs...,/government/organisations/hm-revenue-customs/c...,1.0


In [15]:
occ_shape = df_rel[df_rel.Page_Seq_Occurrences==1].shape[0]

In [36]:
"{}% of user journeys with all related links in 14-16/01 time period occur only once.".format(float('{:.6g}'.
                                                                        format((occ_shape*100)/df_rel.shape[0])))

'85.21% of user journeys with all related links in 14-16/01 time period occur only once.'

### Keep only journeys that are interacting with the "Related content" links

In [40]:
df_relc = df_rel[df_rel.Sequence.str.contains("Related content")].copy(deep=True)

In [42]:
relc_shape = df_relc.shape[0]

In [45]:
"{}% of user journeys include only \"Related content\" links".format(float('{:.4g}'.
                                                                        format((relc_shape*100)/df_rel.shape[0])))

'66.75% of user journeys include only "Related content" links'

In [46]:
df_relc.Page_Seq_Occurrences.describe()

count    394602.000000
mean         16.854377
std         150.269932
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max        5467.000000
Name: Page_Seq_Occurrences, dtype: float64

In [59]:
columns = ['DeviceCategories', 'Dates', 'Page_Event_List', 'Page_List', 
           'Event_List',  'Event_cats_agg', 'Event_cat_act_agg',
           'Taxon_List', 'Taxon_Page_List', 'Page_List_NL']

In [69]:
for column in columns:
    print(column)
    if not isinstance(df_relc[column].iloc[0],list):
        df_relc[column] = df_relc[column].map(literal_eval)

DeviceCategories
Dates
Page_Event_List
Page_List
Event_List
Event_cats_agg
Event_cat_act_agg
Taxon_List
Taxon_Page_List
Page_List_NL


### Compute device category frequency counts over all journeys

In [71]:
devices = Counter()
for item in df_relc.DeviceCategories.values:
    for key,value in item:
        devices[key]+=value